In [ ]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import torch
from torchvision import models, transforms
from PIL import Image
import torch.nn as nn
import pandas as pd
import os
import numpy as np
import glob


In [ ]:
class ResizeToMaxDim:
    def __init__(self, max_size):
        self.max_size = max_size
    
    def __call__(self, img):
        # Get current size
        w, h = img.size
        if w > h:
            new_w = self.max_size
            new_h = int(h * (self.max_size / w))
        else:
            new_h = self.max_size
            new_w = int(w * (self.max_size / h))
        return img.resize((new_w, new_h), Image.LANCZOS)
    
class PadToSquare:
    def __init__(self, size, fill=0):
        self.size = size
        self.fill = fill
    
    def __call__(self, img):
        w, h = img.size
        pad_w = (self.size - w) // 2
        pad_h = (self.size - h) // 2
        padding = (pad_w, pad_h, self.size - w - pad_w, self.size - h - pad_h)
        return transforms.functional.pad(img, padding, fill=self.fill)


def plot_image_with_age(model, image_path, csv_data = None, classification = False, image_size = 448):
    # Define transformation

    
    # transform = transforms.Compose([
    #     transforms.Resize(448),
    #     #transforms.CenterCrop(600),
    #     transforms.ToTensor(),
    #     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    # ])

    transform = transforms.Compose([
            ResizeToMaxDim(image_size),
            PadToSquare(image_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

    # Load and preprocess the image
    if isinstance(image_path, str):
        image = Image.open(image_path).convert('RGB')
    else:
        image = Image.fromarray(image_path)


    input_image_before_cuda = transform(image)
    # plt.imshow(input_image_before_cuda.permute(1, 2, 0))
    # plt.show()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    input_image = input_image_before_cuda.unsqueeze(0).to(device)

    # Get the true age from the CSVnot so clean writtenm since lag_benchmark_cropped is not the best way to make the comparison slower ways I have seen, but couldn come up with similar fast
    if csv_data is not None:
        true_age = csv_data[csv_data['img_name'] == os.path.join("lag_benchmark_cropped",os.path.basename(image_path))]['age'].values[0]
    else:
        true_age = -1

    # Make prediction
    # plt.imshow(input_image_before_cuda.numpy().transpose(1, 2, 0))
    # plt.show()
    with torch.no_grad():
        output = model(input_image)
        if not classification:
            predicted_age = output.item()
        else:
            predicted_age = output
       
    # Plot the image with the predicted and true age
    plt.figure(figsize=(8, 8))
    plt.imshow(image)

    if classification:
        predicted_age_group = nn.Softmax(dim=1)(output)
        # predicted_age_group = np.argmax(output.cpu().numpy())
        # age_group_mapping = {
        #     0: '0-2',
        #     1: '3-13',
        #     2: '18-99',
        # }
        # predicted_age_group = age_group_mapping.get(predicted_age_group, 'Unknown')
        plt.title(f'Estimated Age: {predicted_age_group}\nTrue Age: {true_age}')

    else:
        plt.title(f'Estimated Age: {predicted_age:.2f}\nTrue Age: {true_age}')
    plt.axis('off')
    plt.show()

## Age Regression whole Body

In [ ]:
# Load model weights

model_weights_path = '/usr/users/vhassle/psych_track/AgeSelf/models/age_classification_model_25.pth'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
model_age = models.resnet50(pretrained=False)
num_ftrs = model_age.fc.in_features
model_age.fc = nn.Linear(num_ftrs, 1)
model_age.load_state_dict(torch.load(model_weights_path))
model = model_age.to(device)
model.eval()

# Load true ages from CSV
data = pd.read_csv('/usr/users/vhassle/datasets/lagenda/cropped_data.csv')

In [ ]:
import glob
#image_paths = glob.glob("/usr/users/vhassle/datasets/lagenda/lag_benchmark_cropped/*")
data_subset = data[data['age'] != 100]
image_paths = [os.path.join('/usr/users/vhassle/datasets/lagenda', image_name) for image_name in data_subset['img_name']]

print(len(image_paths))
for image_path in image_paths[2134:2135]:
    plot_image_with_age(model_age, image_path, data)

## Age classification whole Body

In [ ]:
# Load model weights
# model_weights_path = '/usr/users/vhassle/datasets/lagenda/age_classification_model.pth'
model_weights_path = '/usr/users/vhassle/psych_track/AgeSelf/models/age_classification_model_15_focal_pad.pth'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
model_age = models.resnet50(pretrained=False)
num_ftrs = model_age.fc.in_features
model_age.fc = nn.Linear(num_ftrs, 3)
model_age.load_state_dict(torch.load(model_weights_path))
model = model_age.to(device)
model.eval()

# Load true ages from CSV
data_class = pd.read_csv('/usr/users/vhassle/datasets/lagenda/cropped_data_age_classes.csv')

In [ ]:
data_subset = data[data['age'] != 100]
image_paths = [os.path.join('/usr/users/vhassle/datasets/lagenda', image_name) for image_name in data_subset['img_name']]

print(len(image_paths))
for image_path in image_paths[0:100:100]:
    plot_image_with_age(model_age, image_path, data, classification=True)

In [ ]:

#image_paths = glob.glob("/usr/users/vhassle/psych_track/Gender-and-Age-Detection_Face/*.jpg")
image_paths = glob.glob("/usr/users/vhassle/datasets/Wortschatzinsel/cropped_images/*.jpg")
for image_path in image_paths:
    plot_image_with_age(model_age, image_path, None, classification=True)

## Für die Gesichter

In [ ]:

# Load model weights
# model_weights_path = '/usr/users/vhassle/datasets/lagenda/age_classification_model.pth'
model_weights_path = '/usr/users/vhassle/psych_track/AgeSelf/models/faces/age_classification_model_25_focal.pth'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
model_age = models.resnet50(pretrained=False)
num_ftrs = model_age.fc.in_features
model_age.fc = nn.Linear(num_ftrs, 3)
model_age.load_state_dict(torch.load(model_weights_path))
model_age = model_age.to(device)
model_age.eval()
# Load true ages from CSV
data_class = pd.read_csv('/usr/users/vhassle/datasets/lagenda/cropped_data_age_classes.csv')

In [ ]:

#image_paths = glob.glob("/usr/users/vhassle/psych_track/Gender-and-Age-Detection_Face/*.jpg")
#image_paths = glob.glob("/usr/users/vhassle/datasets/Wortschatzinsel/cropped_images/*.jpg")
image_paths = glob.glob("/usr/users/vhassle/datasets/example_images/children/*")
for image_path in image_paths:
    plot_image_with_age(model_age, image_path, None, classification=True, image_size=150)

## combining face detection with age classification in a video

In [ ]:
import cv2
from annotate_videos_functions import process_video
from pytorch_retinaface.detect import process_image, load_Retinanet 

class ResizeToMaxDim:
    def __init__(self, max_size):
        self.max_size = max_size
    
    def __call__(self, img):
        # Get current size
        w, h = img.size
        if w > h:
            new_w = self.max_size
            new_h = int(h * (self.max_size / w))
        else:
            new_h = self.max_size
            new_w = int(w * (self.max_size / h))
        return img.resize((new_w, new_h), Image.LANCZOS)
    
class PadToSquare:
    def __init__(self, size, fill=0):
        self.size = size
        self.fill = fill
    
    def __call__(self, img):
        w, h = img.size
        pad_w = (self.size - w) // 2
        pad_h = (self.size - h) // 2
        padding = (pad_w, pad_h, self.size - w - pad_w, self.size - h - pad_h)
        return transforms.functional.pad(img, padding, fill=self.fill)

def plot_image_with_estimated_ages(image, model_age, model_face_detection ,index_frame = 0, classification=True, image_size=448):
    """
    Plots an image with estimated ages annotated on detected faces.

    Parameters:
    image (Union[str, np.ndarray]): Path to the image or a NumPy array representing the image.
    model: The model used for age estimation.
    classification (bool): If True, classify the age group. If False, predict the exact age.
    image_size (int): The size to which the image should be resized.
    """
    # Define transformation
    transform = transforms.Compose([
        ResizeToMaxDim(image_size),
        PadToSquare(image_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Load the image
    if isinstance(image, str):
        frame = cv2.imread(image)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    elif isinstance(image, np.ndarray):
        frame_rgb = image
        frame = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)
    else:
        raise ValueError("image must be a file path or a numpy array")

    # Detect faces
    faces = process_image(model_face_detection, frame_rgb)
    height, width, _ = frame.shape

    # Prepare for prediction
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    annotations = []

    for face_key in faces.keys():
        face_area = faces[face_key]
        x1, y1, x2, y2 = face_area
        x1, y1, x2, y2 = max(0, x1), max(0, y1), min(width, x2), min(height, y2)

        # Extract the face
        face = frame_rgb[y1:y2, x1:x2]

        # Convert the face to PIL image and apply transformation
        face_pil = Image.fromarray(face)
        input_image_before_cuda = transform(face_pil)
        input_image = input_image_before_cuda.unsqueeze(0).to(device)

        # Make prediction
        with torch.no_grad():
            output = model_age(input_image)
            if not classification:
                predicted_age = output.item()
                predicted_age_text = f'Age: {predicted_age:.2f}'
            else:
                predicted_age_group = nn.Softmax(dim=1)(output)
                age_group = predicted_age_group.argmax(dim=1).item()
                predicted_age_text = f'{age_group}'

        # Annotate the image using OpenCV
        cv2.rectangle(frame_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame_rgb, predicted_age_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Prepare annotation for MOT format
        annotation = [index_frame, face_key, x1, y1, x2 - x1, y2 - y1, 1, -1, -1, -1, age_group]
        annotations.append(annotation)

    return frame_rgb, annotations

In [ ]:


frame = Image.open("/usr/users/vhassle/datasets/example_images/children/1000_F_140125186_ktTYm8WJ8EDK7Fc82g9A9OU3OrTa5cyg.jpg")
frame = np.asarray(frame)
model_face_detection = load_Retinanet("/usr/users/vhassle/psych_track/Pytorch_Retinaface/Resnet50_Final.pth")

model_weights_path = '/usr/users/vhassle/psych_track/AgeSelf/models/faces/age_classification_model_final_focal.pth'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
model_age = models.resnet50(pretrained=False)
num_ftrs = model_age.fc.in_features
model_age.fc = nn.Linear(num_ftrs, 3)
model_age.load_state_dict(torch.load(model_weights_path))
model_age = model_age.to(device)
model_age.eval()



In [ ]:
import matplotlib.pyplot as plt
image_paths = glob.glob("/usr/users/vhassle/datasets/example_images/children/*")
for image_path in image_paths:
    frame, annotations = plot_image_with_estimated_ages(image_path, model_age, model_face_detection ,index_frame = 0, classification=True, image_size=150)
    plt.imshow(frame)
    plt.show()